In [1]:
from pathlib import Path

import click
import numpy as np
import pandas as pd
import xarray as xr
import yaml
from wam2layers.analysis.checks import check_input
from wam2layers.preprocessing.shared import (accumulation_to_flux,
                                                    calculate_humidity,
                                                    insert_level, interpolate,
                                                    sortby_ndarray)

from dask.distributed import Client

In [2]:
client = Client() # n_workers=2, threads_per_worker=2, memory_limit='500MB'

2022-10-21 15:38:57,149 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-w918smo2', purging


In [18]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 7.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41901,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: 3 minutes ago,Total memory: 7.77 GiB
Comm: tcp://127.0.0.1:43879,Total threads: 1
Dashboard: http://127.0.0.1:34665/status,Memory: 7.77 GiB
Nanny: tcp://127.0.0.1:34371,


In [19]:
from wam2layers.preprocessing.era5 import parse_config, preprocess_precip_and_evap, load_data, get_dp_modellevels

In [20]:
config = parse_config('/home/pkalverla/cases/era5_global.yaml')

In [21]:
date = config['datelist'][0]
date

Timestamp('2021-07-01 00:00:00', freq='D')

In [22]:
# 4d fields
levels = config["levels"]
q = load_data("q", date, config)  # in kg kg-1
u = load_data("u", date, config)  # in m/s
v = load_data("v", date, config)  # in m/s
sp = load_data("sp", date, config)  # in Pa

dp = get_dp_modellevels(sp, modellevels=levels)

# Calculate column water vapour
g = 9.80665  # gravitational accelleration [m/s2]
cwv = q * dp / g  # (kg/m2)

# possible correct with tcw
cw = cwv

# Calculate fluxes
fx = u * cw  # eastward atmospheric moisture flux (kg m-1 s-1)
fy = v * cw  # northward atmospheric moisture flux (kg m-1 s-1)

# Vertically integrate over two layers
boundary = 111
idx = dp.level.searchsorted(boundary, side='right')
upper = np.s_[:, :idx, :, :]
lower = np.s_[:, idx:, :, :]

s_lower = cw[lower].sum(dim="level")
s_upper = cw[upper].sum(dim="level")

fx_lower = fx[lower].sum(dim="level")  # kg m-1 s-1
fy_lower = fy[lower].sum(dim="level")  # kg m-1 s-1

fx_upper = fx[upper].sum(dim="level")  # kg m-1 s-1
fy_upper = fy[upper].sum(dim="level")  # kg m-1 s-1

# Load and preprocess precip and evap
precip, evap = preprocess_precip_and_evap(date, config)

# Combine everything into one dataset
ds = xr.Dataset(
    {
        "fx_upper": fx_upper.assign_attrs(units="kg m-1 s-1"),
        "fy_upper": fy_upper.assign_attrs(units="kg m-1 s-1"),
        "fx_lower": fx_lower.assign_attrs(units="kg m-1 s-1"),
        "fy_lower": fy_lower.assign_attrs(units="kg m-1 s-1"),
        "s_upper": s_upper.assign_attrs(units="kg m-2"),
        "s_lower": s_lower.assign_attrs(units="kg m-2"),
        "evap": evap,
        "precip": precip,
    }
)
ds

((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (22,), (641,), (1440,))
((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (22,), (641,), (1440,))
((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (22,), (641,), (1440,))
((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (641,), (1440,))
((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (641,), (1440,))
((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (641,), (1440,))
loaded evap & precip


<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 641, time: 24)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 80.0 79.75 79.5 79.25 ... -79.5 -79.75 -80.0
  * time       (time) datetime64[ns] 2021-07-01 ... 2021-07-01T23:00:00
Data variables:
    fx_upper   (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    fy_upper   (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    fx_lower   (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    fy_lower   (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    s_upper    (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    s_lower    (time, latitude, longitude) float64 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    evap       (time, latitude, longitude) float32 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>
    precip     (time, latitude, longitude) float32 dask.array<chunksize=(1, 641, 1440), meta=np.ndarray>

In [23]:
%%time
ds.to_netcdf('/data/volume_2/test_peter/test5.nc')

2022-10-21 15:42:35,662 - distributed.nanny - WARNING - Restarting worker
2022-10-21 15:42:53,639 - distributed.nanny - WARNING - Restarting worker
2022-10-21 15:43:12,928 - distributed.nanny - WARNING - Restarting worker
2022-10-21 15:43:30,325 - distributed.nanny - WARNING - Restarting worker


KilledWorker: Attempted to run task ('open_dataset-getitem-getitem-b38e1af12b4821a9a12c86ae8db6cc38', 4, 0, 0, 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:33875. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.